# Imports

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import string
import xgboost as xgb
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import tensorflow_hub as hub
import tensorflow as tf

# Carga de los data sets de test y de entrenamiento

In [ ]:
test = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Train_TP2_Datos_2020-2C.csv')

trainSetProcesado = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/train_dataset_procesado.csv')
testSetProcesado = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/test_set_procesado.csv')

testSetProcesado.drop_duplicates(subset="Opportunity_ID", inplace=True)

# Separamos la variable a predecir y los features

In [ ]:
#X = DATA , Y = TARGET 
X1, y1 = trainSetProcesado[['Bureaucratic_Code_0_Approval',
                            'Total_Taxable_Amount_USD',
                            'TRF',
                            'Delivery_Year',
                            'Submitted_for_Approval',
                            'Total_Amount_USD',
                            'ASP',
                            'ASP_(converted)',
                            'Pricing, Delivery_Terms_Approved']], trainSetProcesado['Stage']

Dividimos el set de entrenamiento en train y test, utilizando una proporción del 30% del total de los datos para el segundo.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

# XGBoost

In [ ]:
xg_clas = XGBClassifier(colsample_bytree = 0.5, 
                        learning_rate = 0.27,
                        max_depth = 5, 
                        alpha = 10, 
                        n_estimators = 40)

In [ ]:
xg_clas.fit(X_train,y_train)


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.27, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=40, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:

predicciones_train = xg_clas.predict_proba(X_test)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_train)

0.3998173981723664

In [ ]:
xg_clas.feature_importances_

array([0.08358292, 0.131953  , 0.03031526, 0.14259776, 0.        ,
       0.07409366, 0.17482449, 0.09217028, 0.27046254], dtype=float32)

In [ ]:
predicciones_finales = xg_clas.predict_proba(testSetProcesado[['Bureaucratic_Code_0_Approval',
                            'Total_Taxable_Amount_USD',
                            'TRF',
                            'Delivery_Year',
                            'Submitted_for_Approval',
                            'Total_Amount_USD',
                            'ASP',
                            'ASP_(converted)',
                            'Pricing, Delivery_Terms_Approved']])

In [ ]:
predicciones_exito = []
for prob in predicciones_finales:
  predicciones_exito.append(prob[1])

In [ ]:
csv_kaggle = pd.DataFrame()
csv_kaggle['Opportunity_ID'] = testSetProcesado["Opportunity_ID"]
csv_kaggle['target'] = predicciones_exito

In [ ]:
csv = csv_kaggle.to_csv('predicciones_xgb_multiple_features_02.csv', index=False)
files.download('predicciones_xgb_multiple_features_02.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Resultados

Algoritmo: **XGBoost**

Búsqueda de hiper parámetros: Aleatorio

Features analizados: 

- Total Taxable Amount USD
- Bureaucrotic Code 0 Approval
- TRF
- Delivery_Year
- Submitted_for_Approval
- ASP
- ASP_(converted)
- Pricing, Delivery_Terms_Approved

Hiper parámetros: 

- colsample_bytree = 0.5
- learning_rate = 0.27
- max_depth = 5
- alpha = 10
- n_estimators = 40

**Resultado Kaggle = 0,56805**

## Random Search

In [ ]:
xg_clas = XGBClassifier(colsample_bytree = 0.5, 
                        learning_rate = 0.2,
                        max_depth = 5, 
                        alpha = 10, 
                        n_estimators = 40,
                        eval_metric = 'mlogloss')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

posibles_hiperparametros = {'learning_rate':np.arange(0.1,0.5,0.1),
                              'n_estimators':np.arange(5,115,5),
                              'scale_pos_weight':np.arange(2,6,1),
                              'max_depth':np.arange(4,12,1),
                            'min_child_weight':np.arange(1,10,1),
                              'gamma':np.arange(0,0.5,0.1),
                              'subsample':np.arange(0.6,1,0.1),
                            'colsample_bytree':np.arange(0.6,0.91,0.05),
                              'colsample_bylevel':np.arange(0.6,0.91,0.05)}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

random_hiperparametros = RandomizedSearchCV(xg_clas, 
                                            posibles_hiperparametros, 
                                            n_iter = 20,
                                            verbose = 3,
                                            n_jobs = -1,
                                            scoring=LogLoss)
mejores_hp = random_hiperparametros.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   25.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.9min finished


In [ ]:
mejores_hp.best_params_

{'colsample_bylevel': 0.8500000000000002,
 'colsample_bytree': 0.7000000000000001,
 'gamma': 0.0,
 'learning_rate': 0.30000000000000004,
 'max_depth': 8,
 'min_child_weight': 5,
 'n_estimators': 110,
 'scale_pos_weight': 4,
 'subsample': 0.7999999999999999}

In [ ]:
predicciones_mejores_hp_train = mejores_hp.predict_proba(X_test)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_mejores_hp_train)

0.36366216205259944

In [ ]:
predicciones_mejores_hp = random_hiperparametros.predict_proba(testSetProcesado[['Bureaucratic_Code_0_Approval',
                            'Total_Taxable_Amount_USD',
                            'TRF',
                            'Delivery_Year',
                            'Submitted_for_Approval',
                            'Total_Amount_USD',
                            'ASP',
                            'ASP_(converted)',
                            'Pricing, Delivery_Terms_Approved']])

In [ ]:
predicciones_mejores_hp_exito = []
for prob in predicciones_mejores_hp:
  predicciones_mejores_hp_exito.append(prob[1])

In [ ]:
csv_kaggle_mejores_hp = pd.DataFrame()
csv_kaggle_mejores_hp['Opportunity_ID'] = testSetProcesado["Opportunity_ID"]
csv_kaggle_mejores_hp['target'] = predicciones_mejores_hp_exito

In [ ]:
csv = csv_kaggle_mejores_hp.to_csv('predicciones_xgb_randomSearch_multiple_features.csv', index=False)
files.download('predicciones_xgb_randomSearch_multiple_features.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Resultados

Algoritmo: **XGBoost**

Búsqueda de hiper parámetros: **Random Search**

Features analizados: 

- Bureaucratic_Code_0_Approval
- Total_Taxable_Amount_USD
- TRF
- Delivery_Year
- Submitted_for_Approval
- Total_Amount_USD
- ASP
- ASP_(converted)
- Pricing, Delivery_Terms_Approved

**Resultado Kaggle = 0,6798**

La mejora con respecto a los hiper parámetros propuestos antes de esta búsqueda son demasiado leves, y no hacen diferencia en Kaggle. Probamos haciendo random search con 1000 iteraciones, pero la celda nunca terminaba de correr.